# Business Objectives
This company is the largest online loan marketplace, facilitating personal loans, business loans, and financing of medical procedures. Borrowers can easily access lower interest rate loans through a fast online interface. 

 

Like most other lending companies, lending loans to ‘risky’ applicants is the largest source of financial loss (called credit loss). Credit loss is the amount of money lost by the lender when the borrower refuses to pay or runs away with the money owed. In other words, borrowers who default cause the largest amount of loss to the lenders. In this case, the customers labelled as 'charged-off' are the 'defaulters'. 

 

If one is able to identify these risky loan applicants, then such loans can be reduced thereby cutting down the amount of credit loss. Identification of such applicants using EDA is the aim of this case study.

 

In other words, the company wants to understand the driving factors (or driver variables) behind loan default, i.e. the variables which are strong indicators of default.  The company can utilise this knowledge for its portfolio and risk assessment. 


To develop your understanding of the domain, you are advised to independently research a little about risk analytics (understanding the types of variables and their significance should be enough).

In [82]:
#Import necessary modules
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [83]:
#Load the dataset 
loan=pd.read_csv("loan/loan.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:
#Initial Analysis of dataset
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


info() does not show any information due to too many columns. Need to find irrelevant columns to first remove them.

In [18]:
loan.shape

(39717, 111)

In [10]:
loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [13]:
loan.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=111)

In [54]:
loan.isnull().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
                              ...  
tax_liens                        39
tot_hi_cred_lim               39717
total_bal_ex_mort             39717
total_bc_limit                39717
total_il_high_credit_limit    39717
Length: 111, dtype: int64

In [57]:
#From above result we see there are many columns which contain all NaNs.

In [128]:
loan.shape[0]/2

19858.5

# Probaby good to remove columns containing 50% of NaNs. 

In [136]:
#Print columns containing 50% of NaNs
for col in loan.columns:
  if loan.loc[:, col].isnull().sum() > (loan.shape[0]/2):
    print("Column Name : " + col + " NaN Count " + str(loan.loc[:, col].isnull().sum()))

Column Name : mths_since_last_delinq NaN Count 25682
Column Name : mths_since_last_record NaN Count 36931
Column Name : next_pymnt_d NaN Count 38577
Column Name : mths_since_last_major_derog NaN Count 39717
Column Name : annual_inc_joint NaN Count 39717
Column Name : dti_joint NaN Count 39717
Column Name : verification_status_joint NaN Count 39717
Column Name : tot_coll_amt NaN Count 39717
Column Name : tot_cur_bal NaN Count 39717
Column Name : open_acc_6m NaN Count 39717
Column Name : open_il_6m NaN Count 39717
Column Name : open_il_12m NaN Count 39717
Column Name : open_il_24m NaN Count 39717
Column Name : mths_since_rcnt_il NaN Count 39717
Column Name : total_bal_il NaN Count 39717
Column Name : il_util NaN Count 39717
Column Name : open_rv_12m NaN Count 39717
Column Name : open_rv_24m NaN Count 39717
Column Name : max_bal_bc NaN Count 39717
Column Name : all_util NaN Count 39717
Column Name : total_rev_hi_lim NaN Count 39717
Column Name : inq_fi NaN Count 39717
Column Name : total_

This shows many columns have completely null fields as loan.shape shows 39717 records and above many columns have 39717 null values.

In [121]:
# Good to drop columns containg 30% of NaN.
percentage_nan = 50.00
non_nan_thresh =  int(((100-percentage_nan)/100)*loan.shape[0] + 1) #Calculate 40% of non-NaN values below which all columns wil be dropped
print(non_nan_thresh)

19859


In [131]:
# Drop all columns which have 60% of NaN values
loan_new = loan.dropna( axis=1, 
                thresh=min_count)

In [132]:
#Print columns containing all NaNs
for col in loan_new.columns:
  if loan_new.loc[:, col].isnull().sum() > loan_new.shape[0]/2:
    print("Column Name : " + col + " NaN Count " + str(loan_new.loc[:, col].isnull().sum()))

In [72]:
# Great! - No column found containing only NaNs.

In [133]:
loan_new.isnull().mean() * 100

id                             0.000000
member_id                      0.000000
loan_amnt                      0.000000
funded_amnt                    0.000000
funded_amnt_inv                0.000000
term                           0.000000
int_rate                       0.000000
installment                    0.000000
grade                          0.000000
sub_grade                      0.000000
emp_title                      6.191303
emp_length                     2.706650
home_ownership                 0.000000
annual_inc                     0.000000
verification_status            0.000000
issue_d                        0.000000
loan_status                    0.000000
pymnt_plan                     0.000000
url                            0.000000
desc                          32.580507
purpose                        0.000000
title                          0.027696
zip_code                       0.000000
addr_state                     0.000000
dti                            0.000000


In [120]:
loan_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 54 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          39717 non-null  int64  
 1   member_id                   39717 non-null  int64  
 2   loan_amnt                   39717 non-null  int64  
 3   funded_amnt                 39717 non-null  int64  
 4   funded_amnt_inv             39717 non-null  float64
 5   term                        39717 non-null  object 
 6   int_rate                    39717 non-null  object 
 7   installment                 39717 non-null  float64
 8   grade                       39717 non-null  object 
 9   sub_grade                   39717 non-null  object 
 10  emp_title                   37258 non-null  object 
 11  emp_length                  38642 non-null  object 
 12  home_ownership              39717 non-null  object 
 13  annual_inc                  397

In [100]:
loan_new.desc.isnull().mean()  * 100

32.58050708764509

# Remove irrelevant columns